In [30]:
#import libraries.
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# import keras
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
#import sounddevice as sd
# import tensorflow as tf
# tf.enable_eager_execution()
# from pystoi.stoi import stoi
import h5py
######################
#import libraries.
import matplotlib.pyplot as plt
from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
####import sounddevice as sd
import time
print('imported')
# #######################
inputfile='lilftr_refrmd'
targetfile='lilsingle_dataset_log_16'
# from RBM import rbm_layer
hh = h5py.File(targetfile+'.hdf5', 'r')
d=hh[targetfile]
len_data=d.shape
hh.close()
w=3
# print(len_data)
#######################
#define reconstruct function to reconstruct sound from framed signal.
def reconstruct(wave,angle):
    recon = np.sqrt(np.power(10, wave))
    recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=200, win_length=500, window='hann')
    return recon
#######################
I=0
batch_size=128
epochs_num=50
data_len=len_data[0]

# file = h5py.File('clean_data.hdf5','r')
# val_y = file['clean_data'][0:]
# file.close()
# file = h5py.File('ftr_refrmd_test.hdf5','r')
# val_x = file['ftr_refrmd_test'][0:]
# file.close()
h = [1024,512]
seed = 7
from tensorflow.keras.layers import Activation
np.random.seed(seed)
model = Sequential()
act=layers.LeakyReLU(alpha=0.1)
model.add(Dense(h[0], input_dim = w*len_data[1]))
# model.add(BatchNormalization())
model.add(act)
act=layers.LeakyReLU(alpha=0.1)
model.add(Dense(h[1]))
model.add(act)
act=layers.LeakyReLU(alpha=0.1)
model.add(Dense(len_data[1]))

#############################################
h5f1 = h5py.File(inputfile+'.hdf5','r')
h5f2 = h5py.File(targetfile+'.hdf5','r')
def get_record(i):
    # load the ith record using standard python, return numpy arrays
    Data1 = h5f1[inputfile][i]
    Data2 = h5f2[targetfile][i]
#     print(Data1.shape)
#     print(Data2.shape)
    return (Data1, Data2)

def tf_map_fn(index):
    features, labels = tf.py_function(
        get_record, (index,), (tf.float32, tf.float32))
    features.set_shape((w*len_data[1],))
    labels.set_shape((len_data[1]))
#     do data augmentation here
    return features, labels
# global epochs_num
epoch_size = epochs_num
dataset = tf.data.Dataset.from_tensor_slices((tf.range(epoch_size,)))
is_training=1
if is_training:
    dataset = dataset.repeat().shuffle(epoch_size)
dataset = dataset.map(tf_map_fn)
dataset = dataset.batch(batch_size)
# prefetch data to CPU while GPU processes previous batch
dataset = dataset.prefetch(1)



h5f1_val = h5py.File('ftr_refrmd_test.hdf5','r')
h5f2_val = h5py.File('clean_data.hdf5','r')
# def get_record(i):
#     # load the ith record using standard python, return numpy arrays
#     Data1 = h5f1_val['ftr_refrmd_test'][i]
#     Data2 = h5f2_val['clean_data'][i]
# #     print(Data1.shape)
# #     print(Data2.shape)
#     return (Data1, Data2)

# def tf_map_fn(index):
#     features, labels = tf.py_function(
#         get_record, (index,), (tf.float32, tf.float32))
#     features.set_shape((w*len_data[1],))
#     labels.set_shape((len_data[1]))
# #     do data augmentation here
#     return features, labels
# global epochs_num
epoch_size = epochs_num
dataset_val = tf.data.Dataset.from_tensor_slices((tf.range(epoch_size,)))
is_training=1
if is_training:
    dataset_val = dataset_val.repeat().shuffle(epoch_size)
dataset_val = dataset_val.map(tf_map_fn)
dataset_val = dataset_val.batch(batch_size)
# prefetch data to CPU while GPU processes previous batch
dataset_val = dataset_val.prefetch(1)

############################****************************#######################
# ds=tf.data.Dataset.from_generator(
#         da, (tf.float32, tf.float32))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
steps = len_data[0] // batch_size
history=model.fit( dataset, steps_per_epoch=steps,epochs=epochs_num, verbose=1,validation_data=dataset_val,shuffle=False)
%matplotlib inline
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()
model.save('lilmodel.h5')
# model_json = model.to_json()
# with open("lilmodel_10h_dataset.json", "w") as json_file:
#     json_file.write(model_json)
# model.save_weights("lilmodel_10h_dataset.h5")
# print("Saved model to disk")

imported
Epoch 1/50
 228/5344 [>.............................] - ETA: 11:48 - loss: 0.5258 - acc: 0.5219

KeyboardInterrupt: 

In [12]:
import tensorflow as tf
# import numpy
a=tf.range(10,)


TensorShape([Dimension(10)])

In [27]:
a=tf.constant([1,2,3])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    label_numpy = a.eval()

In [97]:
import h5py 
import tensorflow as tf
hh = h5py.File('ftr_refrmd_10h.hdf5', 'r')
d=hh['ftr_refrmd_10h'][0]
len_data=d.shape
hh.close()
len_data

(771,)

In [95]:
len_data

(771,)

In [92]:
dataset

<PrefetchDataset shapes: ((?, 257), (?,)), types: (tf.float32, tf.float32)>